In [1]:
import pandas as pd
import numpy as np 
import pyodbc
from slugify import slugify
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv(r'C:\Users\bobby\Downloads\rw-sleeper-predictions-2024-04-05.csv')
df['Player_Slug'] = df['Player'].apply(slugify)

In [3]:
for i, row in df.iterrows():
    if row['Market Name'] == 'Rebounds':
        df.loc[i,'line_Type'] ='REB'
    elif row['Market Name'] == '3PT Made':
        df.loc[i,'line_Type'] ='FG3M'
    elif row['Market Name'] == 'Assists':
        df.loc[i,'line_Type'] ='AST'
    elif row['Market Name'] == 'Blocks':
        df.loc[i,'line_Type'] ='BLK'
    elif row['Market Name'] == 'PTS+REB+AST':
        df.loc[i,'line_Type'] ='PTS+REB+AST'
    elif row['Market Name'] == 'Turnovers':
        df.loc[i,'line_Type'] ='TOV'
    elif row['Market Name'] == 'Points':
        df.loc[i,'line_Type'] ='PTS'
    elif row['Market Name'] == 'Steals':
        df.loc[i,'line_Type'] ='STL'

In [4]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

In [5]:

sql = """
SELECT distinct
      [PLAYER_NAME]
      ,PLAYER_ID


  FROM [nba_game_data].[dbo].[PlayerGameLogs]
  where yearSeason=2024
  """
df_actuals = pd.read_sql(sql,cnxn)

In [6]:
df_actuals['Player_Slug'] = df_actuals['PLAYER_NAME'].apply(slugify)


In [7]:
df = pd.merge(df,df_actuals,on='Player_Slug')

In [8]:
df_ =df.iloc[-4:]
df_

,Player,Position,Team,Opponent,Date,Time,Market Name,Line,Site Less,Site More,...,DFS Pick'em Sites Factor,Sportsbooks Factor,RotoWire Projection Factor,Hit Rate Factor,RotoWire Projection,Weighted Hit Rate,Player_Slug,line_Type,PLAYER_NAME,PLAYER_ID
261,Ivica Zubac,C,LAC,UTA,2024-04-05,9:30 pm,PTS+REB+AST,25.5,-137,-122,...,0,0,59.2,-65.0,27.28,24,ivica-zubac,PTS+REB+AST,Ivica Zubac,1627826
262,Ivica Zubac,C,LAC,UTA,2024-04-05,9:30 pm,Points,13.5,-125,-134,...,0,2,20.6,-20.0,14.12,42,ivica-zubac,PTS,Ivica Zubac,1627826
263,Jaden Ivey,G,DET,@MEM,2024-04-05,7:00 pm,Points,19.5,-115,-145,...,16.7,16.9,4.3,-77.5,19.63,19,jaden-ivey,PTS,Jaden Ivey,1631093
264,Jaden Ivey,G,DET,@MEM,2024-04-05,7:00 pm,PTS+REB+AST,27.5,-121,-139,...,0,11.8,51.5,-77.5,29.05,19,jaden-ivey,PTS+REB+AST,Jaden Ivey,1631093


In [15]:
# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
# cursor = cnxn.cursor()
# # Define the structure of the output dataframe based on the example output provided
results_columns = [
    'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION',
       'oppAbrv', 'GAME_ID', 'GAME_DATE', 'overUnder', 'hitRateLast5',
       'hitRateLast10', 'hitRateLast20', 'hitRateSeason', 'hitRateVsOpp',
       'hitRateSum', 'totalHitRateOverFifty', 'hitRateEdgeFactor', 'traded',
       'OpponentPerformanceComparisonMAFlag', 'playerImpactComparisonMAFlag',
       'OppDefensiveRanking', 'OppDefensiveRating',
       'againstTopQuortileDefense', 'againstTopHalfDefense', 'Market Name',
       'Site Less', 'Site More', 'Position'
]

# results_df = pd.DataFrame(columns=results_columns)

# # Assuming `df` is your input dataframe and connection setup is done
# for index, row in df.iterrows():
#     # Extract necessary parameters from the current row
#     threshold = row['Line']
#     player_id = row['PLAYER_ID']
#     game_date = row['Date']

#     # Execute the stored procedure
#     cursor.execute("EXEC usp_GetPlayerGameStats @threshold=?, @player_id=?, @game_date=?", threshold, player_id, game_date)
    
#     # Fetch the results
#     fetched_results = cursor.fetchall()

#     # For each row in fetched_results, append it to the results dataframe
#     for result in fetched_results:
#         result_dict = dict(zip(results_columns, result))
#         results_df = results_df.append(result_dict, ignore_index=True)

# # Close the connection
# cursor.close()
# cnxn.close()

In [9]:
df

,Player,Position,Team,Opponent,Date,Time,Market Name,Line,Site Less,Site More,...,DFS Pick'em Sites Factor,Sportsbooks Factor,RotoWire Projection Factor,Hit Rate Factor,RotoWire Projection,Weighted Hit Rate,Player_Slug,line_Type,PLAYER_NAME,PLAYER_ID
0,Luguentz Dort,F,OKC,@IND,2024-04-05,6:00 pm,PTS+REB+AST,21.5,-132,-127,...,0,-1.9,-96.5,-87.5,18.61,15,luguentz-dort,PTS+REB+AST,Luguentz Dort,1629652
1,Luguentz Dort,F,OKC,@IND,2024-04-05,6:00 pm,Points,14.5,-134,-125,...,0,-6.7,-65.0,-52.5,12.55,29,luguentz-dort,PTS,Luguentz Dort,1629652
2,Luguentz Dort,F,OKC,@IND,2024-04-05,6:00 pm,Rebounds,4.5,-148,-115,...,0,-7,-8.3,-25.0,4.33,40,luguentz-dort,REB,Luguentz Dort,1629652
3,Rudy Gobert,C,MIN,@PHX,2024-04-05,9:00 pm,PTS+REB+AST,27.5,-118,-143,...,0,17.4,100.0,42.5,31.23,67,rudy-gobert,PTS+REB+AST,Rudy Gobert,203497
4,Rudy Gobert,C,MIN,@PHX,2024-04-05,9:00 pm,Rebounds,13.5,-137,-124,...,0,-2.4,39.0,32.5,14.67,63,rudy-gobert,REB,Rudy Gobert,203497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Rayan Rupert,G,POR,@WAS,2024-04-05,6:00 pm,PTS+REB+AST,15.5,-134,-127,...,0,-2,100.0,-82.5,20.35,17,rayan-rupert,PTS+REB+AST,Rayan Rupert,1641712
261,Ivica Zubac,C,LAC,UTA,2024-04-05,9:30 pm,PTS+REB+AST,25.5,-137,-122,...,0,0,59.2,-65.0,27.28,24,ivica-zubac,PTS+REB+AST,Ivica Zubac,1627826
262,Ivica Zubac,C,LAC,UTA,2024-04-05,9:30 pm,Points,13.5,-125,-134,...,0,2,20.6,-20.0,14.12,42,ivica-zubac,PTS,Ivica Zubac,1627826
263,Jaden Ivey,G,DET,@MEM,2024-04-05,7:00 pm,Points,19.5,-115,-145,...,16.7,16.9,4.3,-77.5,19.63,19,jaden-ivey,PTS,Jaden Ivey,1631093


In [25]:
for i, row in df.iterrows():
    process_row(row)
    print(row)
    
process_row(row)

21.5 1629652 2024-04-05 PTS+REB+AST
[]
Player                        Luguentz Dort
Position                                  F
Team                                    OKC
Opponent                               @IND
Date                             2024-04-05
Time                                6:00 pm
Market Name                     PTS+REB+AST
Line                                   21.5
Site Less                              -132
Site More                              -127
Prediction                            -28.3
Lean                                   Less
DFS Pick'em Sites Factor                  0
Sportsbooks Factor                     -1.9
RotoWire Projection Factor            -96.5
Hit Rate Factor                       -87.5
RotoWire Projection                   18.61
Weighted Hit Rate                        15
Player_Slug                   luguentz-dort
line_Type                       PTS+REB+AST
PLAYER_NAME                   Luguentz Dort
PLAYER_ID                           1

KeyboardInterrupt: 

In [24]:
from joblib import Parallel, delayed
import pandas as pd
import pyodbc


# Function to process a single row
def process_row(row):
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    
    # Extracting necessary parameters from the row
    threshold = row['Line']
    player_id = row['PLAYER_ID']
    game_date = row['Date']
    market_name = row['Market Name']
    site_less = row['Site Less']
    site_more = row['Site More']  # Ensure there's no trailing tab or space in your column name
    position = row['Position']
    line_type = row['line_Type']
    print(threshold,player_id,game_date,line_type)
    

    # Execute the stored procedure
    # cursor.execute("{CALL usp_GetPlayerGameStats (?, ?, ?, ?)}", (threshold, player_id, game_date, line_type))

    cursor.execute("EXEC usp_GetPlayerGameStats @threshold=?, @player_id=?, @game_date=?, @line_Type=?", (threshold, player_id, str(game_date),str(line_type)))
    fetched_results = cursor.fetchall()
    print(fetched_results)
    # Close the cursor and connection to avoid resource leaks
    cursor.close()
    cnxn.close()

    # Enhancing each result with additional variables from the input row
    results = []
    for result in fetched_results:
        result_dict = dict(zip(results_columns, result))
        
        # Adding additional variables to the result dictionary
        result_dict['Market Name'] = market_name
        result_dict['Site Less'] = site_less
        result_dict['Site More'] = site_more
        result_dict['Position'] = position

        # Append the enhanced result to the results list
        results.append(result_dict)
    
    return results


# Assuming `df` is your dataframe
# def parallel_process(df):
# Use joblib to parallelize the row processing
results_list = Parallel(n_jobs=-1,verbose=5)(delayed(process_row)(row) for index, row in df.iterrows())

# Flatten the list of lists and create the final dataframe
flat_results = [item for sublist in results_list for item in sublist]
results_df = pd.DataFrame(flat_results)
# return results_df

# Now, you can call the parallel_process function with your dataframe
# results_df = parallel_process(df)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.


KeyboardInterrupt: 

In [12]:
old = pd.read_csv('data.csv').iloc[:,1:]

In [14]:
old.columns

Index(['Unnamed: 0', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION',
       'oppAbrv', 'GAME_ID', 'GAME_DATE', 'overUnder', 'hitRateLast5',
       'hitRateLast10', 'hitRateLast20', 'hitRateSeason', 'hitRateVsOpp',
       'hitRateSum', 'totalHitRateOverFifty', 'hitRateEdgeFactor', 'traded',
       'OpponentPerformanceComparisonMAFlag', 'playerImpactComparisonMAFlag',
       'OppDefensiveRanking', 'OppDefensiveRating',
       'againstTopQuortileDefense', 'againstTopHalfDefense', 'Market Name',
       'Site Less', 'Site More', 'Position'],
      dtype='object')

In [21]:
results_list

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [20]:
results_df

""


In [ ]:
from joblib import Parallel, delayed
import pandas as pd
import pyodbc

# Function to execute stored procedure for a subset of rows
def execute_stored_procedure(sub_df, server, database, results_columns):
    # Establish database connection for the current process
    cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
    cursor = cnxn.cursor()
    
    results_list = []
    for index, row in sub_df.iterrows():
        threshold = row['Line']
        player_id = row['PLAYER_ID']
        game_date = row['Date']
        
        # Execute the stored procedure
        cursor.execute("EXEC usp_GetPlayerGameStats @threshold=?, @player_id=?, @game_date=?", threshold, player_id, game_date)
        
        # Fetch the results
        fetched_results = cursor.fetchall()

        # Add each result to the results_list as a dictionary
        for result in fetched_results:
            result_dict = dict(zip(results_columns, result))
            results_list.append(result_dict)
    
    # Close the database connection
    cursor.close()
    cnxn.close()
    
    return results_list

# Assuming df is your dataframe, and server and database are defined
num_cores = 4 # Or however many cores you want to utilize

# Split the dataframe into chunks and execute the stored procedure in parallel
# Note: Adjust the n_jobs parameter according to your system's capabilities
results = Parallel(n_jobs=-1,verbose=)(
    delayed(execute_stored_procedure)(sub_df, server, database, results_columns)
    for _, sub_df in df.groupby(np.arange(len(df)) // (len(df) / num_cores))
)

# Combine the results into a single dataframe
results_df = pd.DataFrame(columns=results_columns)
for result in results:
    results_df = pd.concat([results_df, pd.DataFrame(result)], ignore_index=True)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.


In [39]:
results_df

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,oppAbrv,GAME_ID,GAME_DATE,overUnder,hitRateLast5,hitRateLast10,hitRateLast20,...,hitRateSum,totalHitRateOverFifty,hitRateEdgeFactor,traded,OpponentPerformanceComparisonMAFlag,playerImpactComparisonMAFlag,OppDefensiveRanking,OppDefensiveRating,againstTopQuortileDefense,againstTopHalfDefense
0,1629048,Goga Bitadze,ORL,ATL,22300821,2024-02-25,0,0.000000,0.000000,0.250000,...,1.520000,1,0,0,1,0,30.0,120.080357,0,0
1,1629048,Goga Bitadze,ORL,ATL,22300821,2024-02-25,0,0.000000,0.000000,0.000000,...,0.000000,0,0,0,1,0,30.0,120.080357,0,0
2,203110,Draymond Green,GSW,DEN,22300824,2024-02-25,0,0.200000,0.100000,0.100000,...,0.420000,0,0,0,0,1,4.0,109.919298,1,1
3,203110,Draymond Green,GSW,DEN,22300824,2024-02-25,1,1.000000,1.000000,1.000000,...,4.990000,5,1,0,0,1,4.0,109.919298,1,1
4,203110,Draymond Green,GSW,DEN,22300824,2024-02-25,1,0.800000,0.800000,0.800000,...,3.550000,4,0,0,0,1,4.0,109.919298,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1630532,Franz Wagner,ORL,TOR,22300963,2024-03-15,1,0.200000,0.300000,0.500000,...,1.940000,1,0,0,1,0,23.0,115.59697,0,0
305,1630532,Franz Wagner,ORL,TOR,22300963,2024-03-15,1,1.000000,1.000000,1.000000,...,5.000000,5,1,0,1,0,23.0,115.59697,0,0
306,1630532,Franz Wagner,ORL,TOR,22300963,2024-03-15,0,0.200000,0.100000,0.200000,...,0.820000,0,0,0,1,0,23.0,115.59697,0,0
307,1630532,Franz Wagner,ORL,TOR,22300979,2024-03-17,1,0.200000,0.200000,0.350000,...,1.530000,0,0,0,1,0,22.0,115.592537,0,0
